# Libraries

In [1]:
import subprocess
import json
import requests
import pandas as pd
import codecs
from datetime import date
today = date.today().strftime("%Y-%m-%d")

# First: this is the part where I authenticate myself to the API 

In [2]:
def get_token():
    cmd = """curl https://api.yusofleet.com/api/v3/back_end_user/authenticate --ssl-no-revoke -H \"Content-Type:application/json\" -d "{\\"saas_office_name\\":\\"Lamma-city\\",\\"email\\":\\"saief@lamma.city\\",\\"password\\":\\"SaiefT511044.\\"}\""""
    result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
    return json.loads(str(result.stdout).split("\'")[1])['jwt']

# Second: here is where we generate the UUID of the data needed

In [3]:
def generate_data_export(JWT, table_name, start, end):
    cmd = """curl -X POST https://api.yusofleet.com/api/v3/back_office/Lamma/Lamma-city/data_exports --ssl-no-revoke -H \"Authorization: Bearer {}\" -H \"Content-Type: application/json\" -d \"{{\\"dataclip_key\\":\\"{}\\", \\"start_date\\": \\"{}\\", \\"end_date\\": \\"{}\\"}}\"""".format(JWT, table_name, start, end)
    result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
    r = json.loads(str(result.stdout).split("\'")[1].replace('\\',''))
    col_name = "uuid" if "uuid" in r else "export_uuid"
    return r[col_name]

# Third: in this part we generate the final URL of the data

In [4]:
def generate_temporary_link(JWT, UUID):
    cmd = """curl -X GET \"https://api.yusofleet.com/api/v3/back_office/Lamma/Lamma-city/data_exports/{}\" --ssl-no-revoke -H \"Authorization: Bearer {}\" """.format(UUID, JWT)
    result = subprocess.run(cmd, stdout=subprocess.PIPE, shell = True)
    return json.loads(str(result.stdout).split("\'")[1])['url'].replace('\\u0026', '&')

# Finally, I combine those previous three functions into one, so I can finally generate the data I need - It will come in a text type

In [12]:
def get_data():
    jwt = get_token()
    uuid = generate_data_export(jwt, "saas_office_connexion_time", '2020-11-05', today)
    url = generate_temporary_link(jwt, uuid)
    r = requests.get(url, allow_redirects=True)
    content = r.content.decode("utf-8-sig","ignore")
    #print(type(content)) #string
    #print(content)
    
    #df = pd.DataFrame(content)
    #df.head()
    
    # the return of this function is a text output 
    # of the data 'connexion time' from 30 october till today
    return content

In [120]:
#get_data()

# And here I turn that row text data into a JSON type so it can be properly used.

In [121]:
def gather_cleaned_data():
    
    data = get_data() 
    lines = data.split('\n')
    #header = lines[0].split(';')
    
    header = ['Id du type de vhicule','Primaire/Secondaire','Working status','Id du chauffeur','Statut','Date','Temps (mn)']    
    df = pd.DataFrame()

    for line in lines[1:] :
        col = line.split(';')
        col = pd.DataFrame(col).transpose()
        
        df = df.append(col)
        
    df.columns = header
    
    return df           

In [123]:
gather_cleaned_data().to_excel(r'C:/Users/Almighty/Python workspace/Lamma/data.xlsx', index = False)

# The following example executes SQL INSERT statement to insert a record into the EMPLOYEE table 

In [17]:
import pymysql


def ExeQr(conn, sql) :
            cur = conn.cursor()
            cur.execute( sql )

conn = pymysql.connect( host='127.0.0.1', user='root', passwd='', db='lamma' )

# CREATE TABLE EMPLOYEE (FIRST_NAME varchar(255),LAST_NAME varchar(255),AGE varchar(255),SEX varchar(255),INCOME varchar(255));
sql = "INSERT INTO EMPLOYEE(FIRST_NAME, LAST_NAME, AGE, SEX, INCOME) VALUES ('MK', 'MXK', 20, 'M', 2000)"

ExeQr(conn, sql)

conn.close()